<a href="https://colab.research.google.com/github/thieuhy/AgenticAI_Business_SJSU/blob/IT-Agent-Capstone-Project/Group_8_IT_Agent_Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install langchain langchain-community langchain-openai langchain-text-splitters faiss-cpu pypdf fastapi uvicorn reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests=

In [8]:
import os
import getpass

print("🔐 Enter your OpenAI API Key below:")
openai_key = getpass.getpass()

os.environ["OPENAI_API_KEY"] = openai_key

print("✅ API key set successfully!")

🔐 Enter your OpenAI API Key below:
··········
✅ API key set successfully!


In [9]:
# ===============================================================
# MULTI-AGENT IT SUPPORT SYSTEM (LangGraph + FAISS + MCP + FastAPI)
# Backend for Lovable Frontend
# ===============================================================

import json
import random
import time
from datetime import datetime
import operator # Import operator for list concatenation
from typing import Dict, List, Literal, Sequence, Annotated, TypedDict
import os # Import os module to handle file system operations
from reportlab.pdfgen import canvas # Import canvas from reportlab

# LangGraph
from langgraph.graph import StateGraph, END

# LangChain messages
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# FastAPI
from fastapi import FastAPI
from pydantic import BaseModel

# Vector Store
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


# ===============================================================
# SHARED GRAPH STATE
# ===============================================================

class ITSupportState(TypedDict):
    user_input: str
    use_case: Literal["phishing", "password_reset", "software_support"]

    # Intake
    request_type: str
    extracted_info: Dict
    priority: str

    # RAG
    retrieved_documents: List[str]
    rag_context: str
    recommendations: List[str]

    # Detection
    threat_level: str
    risk_score: int
    analysis_result: str

    # Workflow
    ticket_id: str
    automated_actions: List[str]
    workflow_status: str

    # Escalation
    needs_escalation: bool
    escalation_team: str
    escalation_id: str

    # Final response
    final_response: str

    # Metadata
    timestamp: str
    messages: Annotated[List[AIMessage], operator.add] # Changed Sequence to List and `list` to `operator.add`


# ===============================================================
# RAG LAYER (FAISS VECTOR STORE)
# ===============================================================

class RAGKnowledgeBase:
    def __init__(self):
        print("Loading RAG Vector Stores...")

        self.embeddings = OpenAIEmbeddings()

        # Load PDFs for each use case (your real files)
        self.vectorstores = {
            "phishing": self.load_pdf("/content/Comprehensive_Phishing_Detection_Guide.pdf"),
            "password_reset": self.load_pdf("/content/Comprehensive_Password_Reset_Guide.pdf"),
            "software_support": self.load_pdf("/content/Comprehensive_Software_Troubleshooting_Guide.pdf"),
        }

    def load_pdf(self, path):
        loader = PyPDFLoader(path)
        docs = loader.load()

        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = splitter.split_documents(docs)

        return FAISS.from_documents(chunks, self.embeddings)

    def query(self, use_case, query_text):
        store = self.vectorstores[use_case]
        results = store.similarity_search(query_text, k=3)
        docs = [doc.page_content for doc in results]
        return docs, "\n\n".join(docs)



    def load_pdf(self, path):
        loader = PyPDFLoader(path)
        docs = loader.load()

        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = splitter.split_documents(docs)

        return FAISS.from_documents(chunks, self.embeddings)

    def query(self, use_case, query_text):
        store = self.vectorstores[use_case]
        results = store.similarity_search(query_text, k=3)
        docs = [doc.page_content for doc in results]
        return docs, "\n\n".join(docs)


rag_system = RAGKnowledgeBase()


# ===============================================================
# MCP LAYER (MOCKED)
# ===============================================================

def mcp_do_action(team, message):
    return f"[MCP] Action executed for {team}: {message}"


# ===============================================================
# AGENTS
# ===============================================================

def intake_agent(state: ITSupportState) -> ITSupportState:
    text = state["user_input"].lower()

    if "phish" in text or "suspicious" in text:
        state["use_case"] = "phishing"
        state["priority"] = "high"
    elif "password" in text or "reset" in text:
        state["use_case"] = "password_reset"
        state["priority"] = "medium"
    else:
        state["use_case"] = "software_support"
        state["priority"] = "medium"

    state["request_type"] = state["use_case"]
    state["extracted_info"] = {"keywords": text.split()}

    return state


def knowledge_agent(state: ITSupportState) -> ITSupportState:
    docs, context = rag_system.query(state["use_case"], state["user_input"])
    state["retrieved_documents"] = docs
    state["rag_context"] = context

    if state["use_case"] == "phishing":
        state["recommendations"] = ["Do not click links", "Forward to IT Security"]
    elif state["use_case"] == "password_reset":
        state["recommendations"] = ["Use reset portal", "Check email for reset link"]
    else:
        state["recommendations"] = ["Restart app", "Update software"]

    return state


def detection_agent(state: ITSupportState) -> ITSupportState:
    if state["use_case"] == "phishing":
        state["threat_level"] = "high"
        state["risk_score"] = 80
        state["analysis_result"] = "Likely phishing attempt."
    else:
        state["threat_level"] = "low"
        state["risk_score"] = 10
        state["analysis_result"] = "Standard IT request."

    return state


def workflow_agent(state: ITSupportState) -> ITSupportState:
    prefix = {"phishing": "SEC", "password_reset": "PWD", "software_support": "SW"}
    state["ticket_id"] = f"{prefix[state['use_case']]}-{random.randint(1000,9999)}"

    state["automated_actions"] = ["Ticket created", "Workflow started"]
    state["workflow_status"] = "In progress"

    return state


def escalation_agent(state: ITSupportState) -> ITSupportState:
    if state["risk_score"] > 50:
        state["needs_escalation"] = True
        state["escalation_team"] = "IT Security Team"
        state["escalation_id"] = f"ESC-{random.randint(10000,99999)}"
        state["final_response"] = mcp_do_action(state["escalation_team"], state["user_input"])
    else:
        state["needs_escalation"] = False
        state["final_response"] = "Issue resolved automatically."

    return state


# ===============================================================
# BUILD LANGGRAPH
# ===============================================================

graph = StateGraph(ITSupportState)
graph.add_node("intake", intake_agent)
graph.add_node("knowledge", knowledge_agent)
graph.add_node("detection", detection_agent)
graph.add_node("workflow", workflow_agent)
graph.add_node("escalation", escalation_agent)

graph.set_entry_point("intake")
graph.add_edge("intake", "knowledge")
graph.add_edge("knowledge", "detection")
graph.add_edge("detection", "workflow")
graph.add_edge("workflow", "escalation")
graph.add_edge("escalation", END)

it_graph = graph.compile()


# ===============================================================
# FASTAPI ENDPOINT FOR LOVABLE
# ===============================================================

app = FastAPI()

class Query(BaseModel):
    text: str

@app.post("/it-support")
def it_support(query: Query):
    state = {
        "user_input": query.text,
        "use_case": "",
        "request_type": "",
        "priority": "",
        "extracted_info": {},
        "retrieved_documents": [],
        "rag_context": "",
        "recommendations": [],
        "threat_level": "",
        "risk_score": 0,
        "analysis_result": "",
        "ticket_id": "",
        "automated_actions": [],
        "workflow_status": "",
        "needs_escalation": False,
        "escalation_team": "",
        "escalation_id": "",
        "final_response": "",
        "timestamp": datetime.now().isoformat(),
        "messages": []
    }

    final_state = it_graph.invoke(state)
    return final_state


Loading RAG Vector Stores...


In [10]:
!pip install pyngrok nest_asyncio

from pyngrok import ngrok
import nest_asyncio
nest_asyncio.apply()

# Add your authtoken
!ngrok config add-authtoken 36d4TVatnBbSMyZ5ONkH1qxSWXz_4SuuVZmdYxxZEgLhiCXqm

# Start the tunnel on port 8000
public_url = ngrok.connect(8000)
public_url


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


<NgrokTunnel: "https://overhotly-unpodded-natalee.ngrok-free.dev" -> "http://localhost:8000">

In [11]:
%%writefile main.py
# your whole FastAPI + LangGraph code here

import json
import random
import time
import operator
from datetime import datetime
from typing import Dict, List, Literal, Annotated, TypedDict

import os

# LangGraph
from langgraph.graph import StateGraph, END

# LangChain messages
from langchain_core.messages import AIMessage

# FastAPI
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

# Vector Store
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


# ===============================================================
# SHARED GRAPH STATE
# ===============================================================

class ITSupportState(TypedDict):
    user_input: str
    use_case: Literal["phishing", "password_reset", "software_support"]

    # Intake
    request_type: str
    extracted_info: Dict
    priority: str

    # RAG
    retrieved_documents: List[str]
    rag_context: str
    recommendations: List[str]

    # Detection
    threat_level: str
    risk_score: int
    analysis_result: str

    # Workflow
    ticket_id: str
    automated_actions: List[str]
    workflow_status: str

    # Escalation
    needs_escalation: bool
    escalation_team: str
    escalation_id: str

    # Final response
    final_response: str

    # Metadata
    timestamp: str
    messages: Annotated[List[AIMessage], operator.add]


# ===============================================================
# RAG LAYER (FAISS VECTOR STORE)
# ===============================================================

class RAGKnowledgeBase:
    def __init__(self):
        print("Loading RAG Vector Stores...")

        self.embeddings = OpenAIEmbeddings()

        # Use your real PDFs – adjust path if needed
        self.vectorstores = {
            "phishing": self.load_pdf("/content/Comprehensive_Phishing_Detection_Guide.pdf"),
            "password_reset": self.load_pdf("/content/Comprehensive_Password_Reset_Guide.pdf"),
            "software_support": self.load_pdf("/content/Comprehensive_Software_Troubleshooting_Guide.pdf"),
        }

    def load_pdf(self, path):
        loader = PyPDFLoader(path)
        docs = loader.load()

        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = splitter.split_documents(docs)

        return FAISS.from_documents(chunks, self.embeddings)

    def query(self, use_case, query_text):
        store = self.vectorstores[use_case]
        results = store.similarity_search(query_text, k=3)
        docs = [doc.page_content for doc in results]
        return docs, "\n\n".join(docs)


rag_system = RAGKnowledgeBase()


# ===============================================================
# MCP LAYER (MOCKED)
# ===============================================================

def mcp_do_action(team, message):
    return f"[MCP] Action executed for {team}: {message}"


# ===============================================================
# AGENTS
# ===============================================================

def intake_agent(state: ITSupportState) -> ITSupportState:
    text = state["user_input"].lower()

    if "phish" in text or "suspicious" in text:
        state["use_case"] = "phishing"
        state["priority"] = "high"
    elif "password" in text or "reset" in text:
        state["use_case"] = "password_reset"
        state["priority"] = "medium"
    else:
        state["use_case"] = "software_support"
        state["priority"] = "medium"

    state["request_type"] = state["use_case"]
    state["extracted_info"] = {"keywords": text.split()}

    return state


def knowledge_agent(state: ITSupportState) -> ITSupportState:
    docs, context = rag_system.query(state["use_case"], state["user_input"])
    state["retrieved_documents"] = docs
    state["rag_context"] = context

    if state["use_case"] == "phishing":
        state["recommendations"] = ["Do not click links", "Forward to IT Security"]
    elif state["use_case"] == "password_reset":
        state["recommendations"] = ["Use reset portal", "Check email for reset link"]
    else:
        state["recommendations"] = ["Restart app", "Update software"]

    return state


def detection_agent(state: ITSupportState) -> ITSupportState:
    if state["use_case"] == "phishing":
        state["threat_level"] = "high"
        state["risk_score"] = 80
        state["analysis_result"] = "Likely phishing attempt."
    else:
        state["threat_level"] = "low"
        state["risk_score"] = 10
        state["analysis_result"] = "Standard IT request."

    return state


def workflow_agent(state: ITSupportState) -> ITSupportState:
    prefix = {"phishing": "SEC", "password_reset": "PWD", "software_support": "SW"}
    state["ticket_id"] = f"{prefix[state['use_case']]}-{random.randint(1000,9999)}"

    state["automated_actions"] = ["Ticket created", "Workflow started"]
    state["workflow_status"] = "In progress"

    return state


def escalation_agent(state: ITSupportState) -> ITSupportState:
    if state["risk_score"] > 50:
        state["needs_escalation"] = True
        state["escalation_team"] = "IT Security Team"
        state["escalation_id"] = f"ESC-{random.randint(10000,99999)}"
        state["final_response"] = mcp_do_action(state["escalation_team"], state["user_input"])
    else:
        state["needs_escalation"] = False
        state["final_response"] = "Issue resolved automatically."

    return state


# ===============================================================
# BUILD LANGGRAPH
# ===============================================================

graph = StateGraph(ITSupportState)
graph.add_node("intake", intake_agent)
graph.add_node("knowledge", knowledge_agent)
graph.add_node("detection", detection_agent)
graph.add_node("workflow", workflow_agent)
graph.add_node("escalation", escalation_agent)

graph.set_entry_point("intake")
graph.add_edge("intake", "knowledge")
graph.add_edge("knowledge", "detection")
graph.add_edge("detection", "workflow")
graph.add_edge("workflow", "escalation")
graph.add_edge("escalation", END)

it_graph = graph.compile()


# ===============================================================
# FASTAPI APP
# ===============================================================

app = FastAPI()

# CORS so Lovable can call this from the browser
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # in prod, replace with your Lovable URL
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class Query(BaseModel):
    text: str

@app.post("/it-support")
def it_support(query: Query):
    state: ITSupportState = {
        "user_input": query.text,
        "use_case": "software_support",  # default, will be overwritten in intake
        "request_type": "",
        "priority": "",
        "extracted_info": {},
        "retrieved_documents": [],
        "rag_context": "",
        "recommendations": [],
        "threat_level": "",
        "risk_score": 0,
        "analysis_result": "",
        "ticket_id": "",
        "automated_actions": [],
        "workflow_status": "",
        "needs_escalation": False,
        "escalation_team": "",
        "escalation_id": "",
        "final_response": "",
        "timestamp": datetime.now().isoformat(),
        "messages": [],
    }

    final_state = it_graph.invoke(state)
    return final_state

Overwriting main.py


In [15]:
!nohup uvicorn main:app --host 0.0.0.0 --port 8000 &> server.log &
print("Server started on port 8000")

Server started on port 8000


In [16]:
from pyngrok import ngrok
import nest_asyncio
nest_asyncio.apply()

public_url = ngrok.connect(8000, "http")
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://overhotly-unpodded-natalee.ngrok-free.dev" -> "http://localhost:8000"


In [17]:
import requests

# 👈 NOTE: add /it-support at the end
url = "https://overhotly-unpodded-natalee.ngrok-free.dev/it-support"

payload = {"text": "I think this is a phishing email about my payroll"}

res = requests.post(url, json=payload)
print("Status:", res.status_code)
print("Body:", res.text[:500])  # just show the first part so we see what comes back


Status: 200
Body: {"user_input":"I think this is a phishing email about my payroll","use_case":"phishing","request_type":"phishing","extracted_info":{"keywords":["i","think","this","is","a","phishing","email","about","my","payroll"]},"priority":"high","retrieved_documents":["Comprehensive IT User Support Guide:\n Phishing Detection & Response\n1. Introduction\nThis guide provides employees with essential information for identifying phishing attempts, responding\ncorrectly to suspicious emails or messages, and pre
